In [1]:
# 필수 데이터 분석 및 시각화, 백테스팅 라이브러리 설치
!pip install pandas numpy plotly bt quantstats requests python-dotenv nbformat openai

## 0. KIS 환경 설정 & 공통 유틸

### 0-1. 환경변수 & 공통 헤더

In [1]:
from pathlib import Path
from datetime import datetime, timedelta

import json
import numpy as np
import pandas as pd
import plotly.express as px
import bt
import quantstats as qs
import requests
from openai import OpenAI

from kis.env import (
    ensure_kis_token,
    KIS_URL_BASE,
    get_api_headers,
    to_float,
    DATA_DIR,
)
from kis.meta import get_etf_name

client = OpenAI()

현재 CSV를 DataFrame으로 읽기

In [4]:
def load_holdings_csv(path: str | Path = "holdings.csv") -> pd.DataFrame:
    """
    ETF 구성 종목 CSV를 표준 컬럼명으로 읽어오는 헬퍼.

    기대하는 원본 컬럼 (한국어):
      - 종목코드
      - 구성종목명
      - 주식수(계약수)
      - 평가금액
      - 시가총액
      - 시가총액 구성비중

    반환 DataFrame 컬럼 (영문):
      - isin
      - name
      - shares
      - valuation
      - market_cap
      - mcap_weight
    """
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"holdings CSV 파일을 찾을 수 없습니다: {path}")

    df = pd.read_csv(path)

    rename_map = {
        "종목코드": "isin",
        "구성종목명": "name",
        "주식수(계약수)": "shares",
        "평가금액": "valuation",
        "시가총액": "market_cap",
        "시가총액 구성비중": "mcap_weight",
    }
    df = df.rename(columns=rename_map)

    # 숫자 컬럼 정리 (쉼표 제거 후 float 변환)
    numeric_cols = ["shares", "valuation", "market_cap", "mcap_weight"]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].map(to_float)

    return df


# 예시: 필요할 때 이런 식으로 호출
# holdings = load_holdings_csv("holdings.csv")
# holdings.head()